# **Aim:**

Build a conversational AI agent that can assist users in booking appointments on your
Google Calendar. The agent should be capable of engaging in a natural, back-and-forth
conversation with the user, understanding their intent, checking calendar availability,
suggesting suitable time slots, and confirming bookings — all seamlessly through chat.

 Install Google API Client

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
from google.colab import files
uploaded = files.upload()  # Upload the downloaded .json key file


Saving promising-saga-464617-c1-7c667a6fa258.json to promising-saga-464617-c1-7c667a6fa258 (10).json


 Setup Calendar API Access

In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import datetime

SERVICE_ACCOUNT_FILE = '/content/promising-saga-464617-c1-7c667a6fa258.json'  # Replace with your actual filename
SCOPES = ['https://www.googleapis.com/auth/calendar']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('calendar', 'v3', credentials=credentials)

calendar_id = 'sameerpathania2@gmail.com'


Test Calendar Access – List Events

In [4]:
now = datetime.datetime.utcnow().isoformat() + 'Z'
events_result = service.events().list(calendarId=calendar_id, timeMin=now,
                                      maxResults=5, singleEvents=True,
                                      orderBy='startTime').execute()
events = events_result.get('items', [])

for event in events:
    print(event['summary'], event['start'])


Interview call(Hub9) between Hub9 and Sameer Pathania {'dateTime': '2025-07-07T10:30:00+05:30', 'timeZone': 'Asia/Kolkata'}


**Install FastAPI**

In [5]:
!pip install fastapi uvicorn nest-asyncio


In [6]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import datetime
from googleapiclient.discovery import build
from google.oauth2 import service_account

app = FastAPI()

# Allow frontend requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

SERVICE_ACCOUNT_FILE = '/content/promising-saga-464617-c1-7c667a6fa258.json'
SCOPES = ['https://www.googleapis.com/auth/calendar']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)
calendar_id = 'sameerpathania2@gmail.com'

class BookingRequest(BaseModel):
    summary: str
    start_time: str  # Format: YYYY-MM-DDTHH:MM:SS
    end_time: str

@app.post("/book")
async def book_slot(req: BookingRequest):
    event = {
        'summary': req.summary,
        'start': {'dateTime': req.start_time, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': req.end_time, 'timeZone': 'Asia/Kolkata'},
    }
    created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
    return {"status": "booked", "event_link": created_event.get('htmlLink')}


Colab doesn't work well with running FastAPI apps via uvicorn.run("calendar_api:app") because it doesn't use actual .py files.

So instead, just define the app in a single cell and run it with uvicorn.run(app) directly.

In [7]:
!pip install fastapi uvicorn nest_asyncio python-multipart


In [8]:
#Define the app directly in the notebook

In [8]:
from fastapi import FastAPI
from pydantic import BaseModel
from google.oauth2 import service_account
from googleapiclient.discovery import build
import nest_asyncio
import uvicorn
import datetime

app = FastAPI()

SERVICE_ACCOUNT_FILE = "/content/promising-saga-464617-c1-7c667a6fa258.json"  # path to uploaded key
SCOPES = ['https://www.googleapis.com/auth/calendar']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)
calendar_id = 'sameerpathania2@gmail.com'  # replace with your calendar ID

class BookingRequest(BaseModel):
    summary: str
    start_time: str
    end_time: str

@app.post("/book")
async def book_event(req: BookingRequest):
    event = {
        'summary': req.summary,
        'start': {'dateTime': req.start_time, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': req.end_time, 'timeZone': 'Asia/Kolkata'},
    }
    event = service.events().insert(calendarId=calendar_id, body=event).execute()
    return {"status": "success", "event_link": event.get('htmlLink')}


This method will run your API live in Colab without needing a separate .py file.

In [10]:
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [15592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15592]


In [9]:
!pip install pyngrok


In [10]:
!ngrok config add-authtoken 2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [35]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("Public URL:", public_url)


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2zLkY1mDUdYNaYRjh5uBnNlFO24, tn_2zLkfBWJZZhvYA8ZV6lzEqkoc6x, tn_2zLkilaIUOaZQzTJs2GUwGtDZip\n\r\n\r\nERR_NGROK_324\r\n"}}


Ensure calendar_api.py File Exists

In [11]:
from fastapi import FastAPI
from pydantic import BaseModel
import datetime

app = FastAPI()

class BookingRequest(BaseModel):
    summary: str
    start_time: datetime.datetime
    end_time: datetime.datetime

@app.post("/book")
def book_event(booking: BookingRequest):
    return {
        "status": "Event booked",
        "summary": booking.summary,
        "start": booking.start_time,
        "end": booking.end_time
    }


 Save the API Code in a Python File (calendar_api.py)

In [12]:
code = '''
from fastapi import FastAPI
from pydantic import BaseModel
import datetime

app = FastAPI()

class BookingRequest(BaseModel):
    summary: str
    start_time: datetime.datetime
    end_time: datetime.datetime

@app.post("/book")
def book_event(booking: BookingRequest):
    return {
        "status": "Event booked",
        "summary": booking.summary,
        "start": booking.start_time,
        "end": booking.end_time
    }
'''
with open("calendar_api.py", "w") as f:
    f.write(code)
print("calendar_api.py saved ✅")


calendar_api.py saved ✅


In [13]:
!pip install fastapi uvicorn nest_asyncio pyngrok


Run the FastAPI Server (DO NOT INTERRUPT)

In [14]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# This will block the cell until you stop it manually
uvicorn.run("calendar_api:app", host="0.0.0.0", port=8000)


INFO:     Started server process [65300]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [65300]


In [24]:
from pyngrok import ngrok

# Set your authtoken first (only once)
ngrok.set_auth_token("2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a")  # Replace with actual token

public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)


🔗 Public URL: NgrokTunnel: "https://b1e9-35-229-100-202.ngrok-free.app" -> "http://localhost:8000"


Create a minimal working FastAPI app (in Colab

Update Your FastAPI App Code

In [15]:
from fastapi import FastAPI
from pydantic import BaseModel
from google.oauth2 import service_account
from googleapiclient.discovery import build

app = FastAPI()

# ✅ Home Endpoint (already working)
@app.get("/")
def home():
    return {"message": "Hello from FastAPI!"}

# ✅ Booking Request Schema
class BookingRequest(BaseModel):
    name: str
    email: str
    start: str  # Format: "2025-07-02T15:00:00"
    end: str    # Format: "2025-07-02T15:30:00"

# ✅ Load service account credentials
SERVICE_ACCOUNT_FILE = "/content/promising-saga-464617-c1-7c667a6fa258.json"  # update if needed
SCOPES = ['https://www.googleapis.com/auth/calendar']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

calendar_id = "sameerpathania2@gmail.com"

# ✅ Booking Endpoint
@app.post("/book")
def book_meeting(data: BookingRequest):
    service = build("calendar", "v3", credentials=credentials)

    event = {
        'summary': f"Meeting with {data.name}",
        'description': f"Booked by {data.email}",
        'start': {'dateTime': data.start, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': data.end, 'timeZone': 'Asia/Kolkata'},
    }

    created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
    return {
        "status": "Booked",
        "eventLink": created_event.get("htmlLink")
    }


In [15]:
# ✅ Install once
!pip install fastapi uvicorn nest_asyncio pyngrok

# ✅ Import modules
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pyngrok import ngrok
import threading

# ✅ Apply asyncio patch (needed for Colab)
nest_asyncio.apply()

# ✅ Define FastAPI app
app = FastAPI()

@app.get("/")
def home():
    return {"message": "Hello from FastAPI!"}

# ✅ Start FastAPI server in a background thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()

# ✅ Start ngrok and expose FastAPI server
ngrok.set_auth_token("2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a")  # 🔁 Replace this
public_url = ngrok.connect(8000)

print("🚀 Open Swagger UI at:", public_url.public_url + "/docs")


INFO:     Started server process [63386]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Open Swagger UI at: https://e7a3-35-229-100-202.ngrok-free.app/docs


minimal complete version of your FastAPI app with /book defined correctly

In [16]:
from fastapi import FastAPI
from pydantic import BaseModel
from google.oauth2 import service_account
from googleapiclient.discovery import build

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Hello from FastAPI!"}

class BookingRequest(BaseModel):
    name: str
    email: str
    start: str  # e.g. "2025-07-02T16:00:00"
    end: str    # e.g. "2025-07-02T16:30:00"

# ✅ Load credentials
SERVICE_ACCOUNT_FILE = "/content/promising-saga-464617-c1-7c667a6fa258.json"  # Upload via Colab
SCOPES = ['https://www.googleapis.com/auth/calendar']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_id = "sameerpathania2@gmail.com"


@app.post("/book")
def book_event(data: BookingRequest):
    service = build("calendar", "v3", credentials=credentials)

    event = {
        'summary': f"Meeting with {data.name}",
        'description': f"Booked by {data.email}",
        'start': {'dateTime': data.start, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': data.end, 'timeZone': 'Asia/Kolkata'},
    }

    created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
    return {"status": "Booked", "eventLink": created_event.get("htmlLink")}


Re-run FastAPI Server in Colab

In [17]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


INFO:     Started server process [65300]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [19]:
from pyngrok import ngrok
ngrok.set_auth_token("2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a")
public_url = ngrok.connect(8000)
print(public_url.public_url + "/docs")


https://aca7-35-229-100-202.ngrok-free.app/docs


In [24]:
!pip install fastapi nest_asyncio pyngrok uvicorn google-api-python-client google-auth --quiet

import nest_asyncio
import uvicorn
import threading
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.colab import files

# Step 1: Upload your service account key file
files.upload()  # Upload your JSON key here

# Step 2: Fill these three values correctly
SERVICE_ACCOUNT_FILE = "/content/promising-saga-464617-c1-7c667a6fa258.json"  # <-- CHANGE THIS!
CALENDAR_ID = "sameerpathania2@gmail.com"          # <-- CHANGE THIS!
NGROK_AUTH_TOKEN = "2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a"            # <-- CHANGE THIS!

# Step 3: Setup FastAPI app
app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello from FastAPI!"}

class BookingRequest(BaseModel):
    name: str
    email: str
    start: str  # ISO Format: "2025-07-02T15:00:00"
    end: str

# Step 4: Load credentials
SCOPES = ['https://www.googleapis.com/auth/calendar']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

# Step 5: POST /book endpoint
@app.post("/book")
def book_event(data: BookingRequest):
    service = build("calendar", "v3", credentials=credentials)
    event = {
        'summary': f"Meeting with {data.name}",
        'description': f"Booked by {data.email}",
        'start': {'dateTime': data.start, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': data.end, 'timeZone': 'Asia/Kolkata'},
    }
    created_event = service.events().insert(calendarId=CALENDAR_ID, body=event).execute()
    return {"status": "Booked", "eventLink": created_event.get("htmlLink")}

# Step 6: Run FastAPI server
nest_asyncio.apply()
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()

# Step 7: Launch ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8000)
print("🚀 Swagger UI:", public_url.public_url + "/docs")


Saving promising-saga-464617-c1-7c667a6fa258.json to promising-saga-464617-c1-7c667a6fa258 (11).json


INFO:     Started server process [65300]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🚀 Swagger UI: https://ac30-35-229-100-202.ngrok-free.app/docs


Don't run app = FastAPI() in one cell, and the @app.post("/book") in another.
This clears the routes!

✅ Instead, everything — app = FastAPI(), @app.get("/"), @app.post("/book") — must be in the same cell, like this:

In [18]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Hello from FastAPI!"}

class BookingRequest(BaseModel):
    name: str
    email: str
    start: str
    end: str

@app.post("/book")
def book_event(data: BookingRequest):
    return {"status": "booked"}


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

In [27]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


INFO:     Started server process [71212]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [26]:
from pyngrok import ngrok
ngrok.set_auth_token("2ytVCLf8CM6OIJv6wAakszylWPu_5nmyStXMYycWWiXUoJ49a")  # Replace with your token
public_url = ngrok.connect(8000)
print("Swagger UI:", public_url.public_url + "/docs")


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2zLnX8SglOrbe4l3PwQODeB1gti, tn_2zLo1jW2GEA2JORQI6y37DLQT35, tn_2zLo9XCqX8Zn8wAQIHmcuLKQ0cZ\n\r\n\r\nERR_NGROK_324\r\n"}}


In [19]:
from fastapi import HTTPException

@app.post("/book")
def book_event(data: BookingRequest):
    print("📥 Incoming booking request:", data)

    try:
        # Step 1: Build Event
        event = {
            "summary": f"Meeting with {data.name}",
            "description": f"Booked by {data.email}",
            "start": {"dateTime": data.start, "timeZone": "Asia/Kolkata"},
            "end": {"dateTime": data.end, "timeZone": "Asia/Kolkata"},
        }
        print("📅 Event to be inserted:", event)

        # Step 2: Insert event
        created_event = service.events().insert(calendarId=CALENDAR_ID, body=event).execute()
        print("✅ Event created:", created_event)

        return {"status": "Booked", "eventLink": created_event.get("htmlLink")}

    except Exception as e:
        print("❌ Internal Server Error:", e)
        raise HTTPException(status_code=500, detail=str(e))


In [23]:
!pip install streamlit


In [20]:
# streamlit_app.py
import streamlit as st
import requests
from datetime import datetime, timedelta
import re

st.title("🗓️ Google Calendar Booking Assistant")

st.markdown("Ask me to book a meeting like: *Book Sameer for tomorrow at 3PM to 4PM*")

user_input = st.text_input("🗣️ Your request")

def extract_booking_info(text):
    name_match = re.search(r"book\s+([A-Za-z]+)", text, re.IGNORECASE)
    time_match = re.findall(r"(\d{1,2})(?:[:.]?(\d{2})?)?\s*(AM|PM)?", text, re.IGNORECASE)
    if name_match and len(time_match) >= 2:
        name = name_match.group(1)
        start_hour = int(time_match[0][0])
        end_hour = int(time_match[1][0])
        am_pm = time_match[0][2] or 'PM'

        if am_pm.upper() == 'PM' and start_hour < 12:
            start_hour += 12
            end_hour += 12

        tomorrow = datetime.now() + timedelta(days=1)
        start = tomorrow.replace(hour=start_hour, minute=0, second=0, microsecond=0)
        end = tomorrow.replace(hour=end_hour, minute=0, second=0, microsecond=0)

        return {
            "name": name,
            "email": f"{name.lower()}@example.com",
            "start": start.isoformat(),
            "end": end.isoformat()
        }
    return None

if user_input:
    data = extract_booking_info(user_input)
    if data:
        response = requests.post("https://<your-ngrok-url>.ngrok-free.app/book", json=data)
        if response.status_code == 200:
            event_link = response.json().get("eventLink")
            st.success(f"✅ Booking confirmed for {data['name']}!\n\n[Click here to view event]({event_link})")
        else:
            st.error("❌ Booking failed. Try again.")
    else:
        st.warning("⚠️ Could not extract booking details. Try again.")


2025-07-03 05:24:22.280 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.734 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-03 05:24:22.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.741 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.757 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 05:24:22.771 Thread 'MainThread': mi

In [25]:
!pip install streamlit pyngrok

# Save your Streamlit app
with open("streamlit_app.py", "w") as f:
    f.write("""
import streamlit as st
st.title("Google Calendar Booking App")
st.write("Streamlit is working inside Colab!")
""")

# Create a tunnel
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Streamlit app URL:", public_url)

# Run Streamlit
!streamlit run streamlit_app.py &>/dev/null &


Streamlit app URL: NgrokTunnel: "https://68db-35-229-100-202.ngrok-free.app" -> "http://localhost:8501"


In [30]:

import streamlit as st
import requests
from datetime import datetime, timedelta

st.title("📅 Book a Google Calendar Appointment")

with st.form("booking_form"):
    name = st.text_input("Your Name", value="Sameer")
    email = st.text_input("Your Email", value="sameerpathania2@example.com")

    date = st.date_input("Select Date", datetime.now().date())
    start_time = st.time_input("Start Time", datetime.now().time())
    end_time = st.time_input("End Time", (datetime.now() + timedelta(minutes=30)).time())

    submit = st.form_submit_button("Book Now")

if submit:
    start = datetime.combine(date, start_time)
    end = datetime.combine(date, end_time)

    data = {
        "name": name,
        "email": email,
        "start": start.isoformat(),
        "end": end.isoformat()
    }

    # Replace with your actual ngrok URL
    url = " https://2718-35-229-100-202.ngrok-free.app/book"  # 👈 Replace this with your live /book URL

    try:
        response = requests.post(url, json=data)
        result = response.json()
        if response.status_code == 200:
            st.success("✅ Booking successful!")
            st.markdown(f"[Click to View Event 📆]({result['eventLink']})")
        else:
            st.error(f"❌ Failed: {result}")
    except Exception as e:
        st.error(f"Error: {e}")



2025-07-03 04:16:20.321 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 04:16:20.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [21]:
with open("streamlit_app.py", "w") as f:
    f.write("""
import streamlit as st
import requests
from datetime import datetime, timedelta

st.title("📅 Book a Google Calendar Appointment")

with st.form("booking_form"):
    name = st.text_input("Your Name", value="Sameer")
    email = st.text_input("Your Email", value="sameer@example.com")

    date = st.date_input("Select Date", datetime.now().date())
    start_time = st.time_input("Start Time", datetime.now().time())
    end_time = st.time_input("End Time", (datetime.now() + timedelta(minutes=30)).time())

    submit = st.form_submit_button("Book Now")

if submit:
    start = datetime.combine(date, start_time)
    end = datetime.combine(date, end_time)

    data = {
        "name": name,
        "email": email,
        "start": start.isoformat(),
        "end": end.isoformat()
    }

    url = "https://your-ngrok-url.ngrok-free.app/book"  # Replace with your actual FastAPI endpoint

    try:
        response = requests.post(url, json=data)
        result = response.json()
        if response.status_code == 200:
            st.success("✅ Booking successful!")
            st.markdown(f"[Click to View Event 📅]({result['eventLink']})")
        else:
            st.error(f"❌ Failed: {result}")
    except Exception as e:
        st.error(f"Error: {e}")
""")


In [22]:
!pip install streamlit pyngrok


# **FastAPI Backend with /book Endpoint**

In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from google.oauth2 import service_account
from googleapiclient.discovery import build
from fastapi.middleware.cors import CORSMiddleware
import datetime

app = FastAPI()

# CORS setup
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request model
class BookingRequest(BaseModel):
    name: str
    email: str
    start: str
    end: str

@app.get("/")
def root():
    return {"message": "Hello from FastAPI!"}

@app.post("/book")
def book_appointment(req: BookingRequest):
    creds = service_account.Credentials.from_service_account_file(
        "service_account.json",
        scopes=["https://www.googleapis.com/auth/calendar"]
    )
    service = build("calendar", "v3", credentials=creds)

    event = {
        "summary": f"Meeting with {req.name}",
        "description": f"Auto-scheduled meeting with {req.email}",
        "start": {"dateTime": req.start, "timeZone": "Asia/Kolkata"},
        "end": {"dateTime": req.end, "timeZone": "Asia/Kolkata"},
        "attendees": [{"email": req.email}],
    }

    calendar_id = "primary"
    event = service.events().insert(calendarId=calendar_id, body=event, sendUpdates="all").execute()

    return {
        "status": "Booked",
        "eventLink": event.get("htmlLink")
    }


# **streamlit_app.py – Streamlit Frontend**

In [ ]:
import streamlit as st
import requests
from datetime import datetime, timedelta

st.title("📅 Google Calendar Booking App")
st.write("Fill in the details below to schedule a meeting.")

with st.form("booking_form"):
    name = st.text_input("Your Name", value="Sameer")
    email = st.text_input("Your Email", value="sameer@example.com")

    date = st.date_input("Select Date", datetime.now().date())
    start_time = st.time_input("Start Time", datetime.now().time())
    end_time = st.time_input("End Time", (datetime.now() + timedelta(minutes=30)).time())

    submitted = st.form_submit_button("Book Appointment")

if submitted:
    start = datetime.combine(date, start_time)
    end = datetime.combine(date, end_time)

    payload = {
        "name": name,
        "email": email,
        "start": start.isoformat(),
        "end": end.isoformat()
    }

    try:
        response = requests.post("https://your-ngrok-url.ngrok-free.app/book", json=payload)  # Replace with your ngrok
        result = response.json()
        if response.status_code == 200:
            st.success("✅ Appointment Booked!")
            st.markdown(f"[📅 View Event]({result['eventLink']})")
        else:
            st.error("❌ Booking failed.")
    except Exception as e:
        st.error(f"Error: {e}")
